# Indexing 
- Trong mô hình RAG cơ bản, quá trình indexing là quá trình chuyển đổi một bộ dữ liệu, tài liệu lớn, được chia nhỏ thành các phần, được index vào trong các vectordatabase. Trong phần này sẽ trình bày các kĩ thuật xử lý trong quá trình indexing nhằm nâng cao hiệu quả của trích xuất. Từ việc xử lý với các đoạn văn bản dài, các phân chia (chunking) sao cho phù hợp,.... 

<p align="center">
    <img src="../doc/image/indexing.png" alt="basic-pipeline" width="400"/>
</p>


# Setup 

In [2]:
# setup các biến môi trường
import os 
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# Multi-representation indexing 
- Thông thường, dữ liệu thường được phần chia và split thành nhiều phần khác nhau với size được fix sẵn. Tuy nhiên khi trích xuất thông tin các phần dữ liệu được fix sẵn này tỏ ra hoạt động không hiệu quả. Từ đó, theo nghiên cứu, người ta thay vì trực tiếp embedding các phần dữ liệu nhỏ này thì người ta thường embedding proposition của nó theo [bài báo](https://arxiv.org/pdf/2312.06648.pdf) và trích xuất toàn bộ tài liệu liên quan vào trong các LLM để xử lý. 

<p align="center">
    <img src="../doc/image/multi-presentation-indexing.png" alt="basic-pipeline" width="400"/>
</p>

- Các này làm cho dữ liệu trích xuất ra không chỉ có chữ mà còn có thể là hình ảnh, dạng bảng, .. 

- Dựa vào pipeline trên, để sinh proposition, đoạn các đoạn thông tin sẽ được chia nhỏ theo từng đoạn dài (theo trang hoặc nhiều trang) -> tóm tắt chúng và embedding nó vào trong vector store.
- Sau dó, các câu hỏi của người dùng sẽ được embedding vào vector store và so sánh các câu hỏi từ đó. Kết quả đầu ra sẽ là toàn bộ phần document từ việc so sánh các vectorstore 

In [3]:
# chuẩn bị dữ liệu 
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

loader = WebBaseLoader("https://lilianweng.github.io/posts/2024-02-05-human-data-quality/")
docs.extend(loader.load())

In [6]:
# tạo sinh proposition index 

from langchain_core.documents import Document 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = 'gemini-1.5-pro-latest', temperature = 0.2, api_key = GOOGLE_API_KEY)

# sinh tóm tắt cho một trang của document 

summarize_chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | llm 
    | StrOutputParser()
)


summaries = summarize_chain.batch(docs, {"max_concurrency": 5})


In [ ]:
from langchain.storage import InMemoryByteStore 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma 
from langchain.retrievers.multi_vector import MultiVectorRetriever
import uuid 
# init vector store 
vectorstore = Chroma(collection_name="summaries", 
                     embedding_function= GoogleGenerativeAIEmbeddings(model="models/embedding-001",  google_api_key = GOOGLE_API_KEY))

# xây dựng index giữa tóm tắt và document 
store = InMemoryByteStore()
id_key = "doc_id"

# retriever 
retriever = MultiVectorRetriever(
    vectorstore = vectorstore, 
    byte_store = store, 
    id_key = id_key 
)
doc_id = [str(uuid.uuid4()) for _ in docs]

# link doc with summaries 
summary_docs = [
    Document(page_content = s, metadata = {id_key : doc_id[i]}) for i, s in enumerate(summaries)
]

# add 
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_id, docs)))

In [ ]:
# trich xuat cau summary gan nhat trong vctdb 
query = "Memory in agents"
sub_docs = vectorstore.similarity_search(query, k = 1)

# trich xuat tai lieu lien quan 
retrieval_docs = retriever.get_releavant_documents(query, n_results = 1)


# RAPTOR 
- Với một số câu hỏi, yêu cầu LLM cần phải hiểu rõ bộ tài liệu thì mới có thể trả lời được -> cần có long context. Tuy nhiên, việc xử lý long context với các LLM hiện đại khá đơn giản nhưng có độ trễ cao và tương đối tốn kém. Do đó một giải pháp đó là người ta đã thiết kế mô hình RAPTOR 
- Kiến trúc của RAPTOR được thiết kế theo dạng hình cây, với node lá là document gốc, các document sẽ lần lượt được nhóm lại (clustering) sau đó tóm tắt thành những văn bản tổng quát hơn và lại lần lượt được embedding vào trong cơ sở dữ liệu 


<p align="center">
    <img src="../doc/image/raptor.png" alt="basic-pipeline" width="400"/>
</p>